In [3]:
#Preamble
import ipywidgets as widgets
import io
import os
import pandas as pd
from matplotlib.pyplot import *
import rpy2
import rpy2.robjects as robjects
import numpy
import textwrap



import census_helpers

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)



### Introduction
This notebook will help you answer the question of "Does a certain kind of police activity happen disproportionately (as compared to the town's population) to different racial groups?"

### **Step #0:** Lets get some census data to compare to

If you have a CSV with the racial makeup of your town, upload that. Otherwise, we (eventually) will automatically pull census data.

Usually this can easily be found by going to http://data.census.gov and typing your town's name in the search bar.  Then, select the 'RACE' data file, and click 'CSV' to download a CSV.

In [ ]:
outputWidget0 = widgets.Output(layout={'border': '1px solid black'})
outputWidget1 = widgets.Output(layout={'border': '1px solid black'})
display(outputWidget0)
display(outputWidget1)
cdf = census_helpers.censusDataFetcher(outputWidget0,outputWidget1)
townRace=cdf.townRace



### **Step #1:** Load the police data

Upload a CSV file containing data on police encounters here. We assume each row is a unique encounter with police.

In [4]:
outputWidget = widgets.Output(layout={'border': '1px solid black'})

fileUpload1 = widgets.FileUpload()
display(fileUpload1)

outputWidget3 = widgets.Output(layout={'border': '1px solid black'})
raceDropDown=widgets.Dropdown()
def load_data():
    global arrests
    arrests = pd.read_csv(io.BytesIO(fileUpload1.value[-1].content),index_col=[0])
    global raceDropDown
    raceDropDown.options=arrests.columns
    with outputWidget3:
        display(raceDropDown)
    
    with outputWidget:
        display("Successfully loaded the datafile " + fileUpload1.value[-1].name )

fileUpload1.observe(lambda b: load_data(), names='value')
display(outputWidget)

Dropdown(options=('Durham_arrests_charges.csv', 'Durham_census_race_table.csv'), value='Durham_arrests_charges…

Button(description='Load data file', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

<IPython.core.display.Javascript object>

What kind of encounter does this datafile represent (e.g. arrest, traffic stop, etc.)?

In [ ]:
encounterType = widgets.Text( description='Encounter type:', style= {'description_width': 'initial'})
display(encounterType)

If you would like to preview your data, select how many rows you would like to preview from the dropdown.

In [5]:
outputWidget2 = widgets.Output(layout={'border': '1px solid black'})
# Take a first look:
numRows = widgets.Dropdown(
    options=['5', '10', '15', '20'],
    description='Number:',
    disabled=False,
)
display(numRows)
def preview_data():
    outputWidget2.clear_output()
    with outputWidget2:
        display(arrests.sample(int(numRows.value)))
numRows.observe(lambda b: preview_data(), names='value')
display(outputWidget2)

Dropdown(description='Number:', options=('5', '10', '15', '20'), value='5')

Button(description='Preview', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

### **Step #2:** Which column has the racial data?

Which of these columns represents the race of the person described in the police report? 

In [ ]:
%matplotlib inline
display(outputWidget3)

#For later use
policeRaceSelectMultiple = widgets.SelectMultiple()
def raceDropdownChange():
    policeRaceSelectMultiple.options = arrests.groupby(raceDropDown.value).groups.keys()
    with outputWidget4:
        display(policeRaceSelectMultiple)
    with outputWidget3:
        arrests[raceDropDown.value].value_counts(normalize=True).plot.bar()
        show()
raceDropDown.observe(lambda b: raceDropdownChange(),names='value')

In order to compare, we need to somehow match up Census racial categories with police racial categories.
Select a census racial category from the dropdown, and then select which categories from the police data should match to it.
When finished, click the button to confirm.

In [ ]:
#May refactor later. Here is a first go
outputWidget4 = widgets.Output(layout={'border': '1px solid black'})
censusRaceDropDown = widgets.Dropdown()
censusRaceDropDown.options = townRace['Race'].values.tolist()
with outputWidget4:
    display(censusRaceDropDown)

#Make a dropdown for race categories from census
raceMapDict={}
def policeSelectChange():
    global raceMapDict
    with outputWidget4:
        if censusRaceDropDown.value:
            raceMapDict[censusRaceDropDown.value]=policeRaceSelectMultiple.value
            
            if policeRaceSelectMultiple.value:
                display("Police categories " + " and ".join(policeRaceSelectMultiple.value) + " are now mapped to " + censusRaceDropDown.value)
            else:
                display("No police categories are mapped to "+censusRaceDropDown.value)
        else:
            display("Select a census category first")

policeRaceSelectMultiple.observe(lambda b: policeSelectChange(), names='value')

display(outputWidget4)

In [ ]:
outputWidget5 = widgets.Output(layout={'border': '1px solid black'})

dictButton = widgets.Button(
    description='Click to confirm mapping of categories',
)

display(dictButton)

def displayDict():
    outputWidget5.clear_output()
    with outputWidget5:
        #Check if they accidentally mapped a category multiple places
        for x in set([ item for l in raceMapDict.values() for item in l]):
            if [ item for l in raceMapDict.values() for item in l].count(x)> 1:
                display("Warning: " + str(x) + " is mapped to multiple categories")
        #Display the mapping for verification
        for key in raceMapDict.keys():
            display(key +" is mapped to " + " and ".join(raceMapDict[key]))

        X = raceMapDict.keys()
        Ytown=[]
        Ypolice=[]
        townRace.set_index('Race', inplace = True)
        for key in X:
            Ytown.append(townRace.loc[key,'Percentage'])
            Ypolice.append(sum([arrests[raceDropDown.value].value_counts(normalize=True)[v] for v in raceMapDict[key]]))

        X_axis = numpy.arange(len(X))

        if encounterType.value:
            encounterString = encounterType.value
        else:
            encounterString = "Police encounters"
        
        matplotlib.pyplot.bar(X_axis - 0.2, Ytown, 0.4, label = 'Town')
        matplotlib.pyplot.bar(X_axis + 0.2, Ypolice, 0.4, label = encounterString)
        
        #Handle long category names gracefully
        f = lambda x: textwrap.fill(x, 10)
        matplotlib.pyplot.xticks(X_axis, map(f,X))
        matplotlib.pyplot.xlabel("Race")
        matplotlib.pyplot.ylabel("Percentage")
        matplotlib.pyplot.title("Comparison of town demographics to " + encounterString)
        matplotlib.pyplot.legend()
        matplotlib.pyplot.show()
dictButton.on_click(lambda b: displayDict())


display(outputWidget5)